# union find

In [ ]:
import numpy as np


class Go:
    def __init__(self):
        self.parent = np.zeros(5 * 5, dtype=np.int8) # parent of each stone
        self.stones = [set() for _ in range(5 * 5)] # set of which stones are in the same group of a root
        self.colors = np.zeros(5 * 5, dtype=np.int8)
        self.liberties = np.zeros(5 * 5, dtype=np.int8)

        self.state = np.zeros((5, 5), dtype=np.int8)

    def encode_action(self, x: int, y: int) -> int:
        return x * 5 + y

    def decode_action(self, action_idx: int):
        x = action_idx // 5
        y = action_idx % 5
        return (x, y)

    def find(self, i: int) -> int:
        if self.parent[i] == i:
            return i
        j = self.find(self.parent[i])
        self.parent[i] = j
        return j

    def union(self, a: tuple[int, int], b: tuple[int, int]) -> None:
        self.parent[self.find(self.encode_action(a[0], a[1]))] = self.find(
            self.encode_action(b[0], b[1])
        )

    def find_group_of_stone(self, x: int, y: int) -> set[tuple[int, int]]:
        i = self.encode_action(x, y)
        j = self.find(i)
        return self.decode_action(j)

In [ ]:
go = Go()
go.find_group_of_stone(0, 0)

(0, 0)